In [1]:
import pandas as pd
# #!pip install geocoder
# !pip install gensim
import geocoder
import numpy as np
import gensim
import os
import string
import re
import pprint
from gensim import corpora, models, similarities
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
from sklearn.manifold import TSNE
import nltk
from nltk import word_tokenize, tokenize
nltk.download('punkt')

%matplotlib inline
from matplotlib import pyplot as plt
#!pip install vaderSentiment
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
import seaborn as sns
import datetime
from collections import Counter

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\CK\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [2]:
trump_pd = pd.read_csv('../data/hashtag_donaldtrump.csv', engine='python')
trump_df = trump_pd[trump_pd['country'] == 'United States of America']
trump_df.shape

(178897, 21)

In [4]:
biden_pd = pd.read_csv('../data/hashtag_joebiden.csv', engine='python')
biden_df = biden_pd[biden_pd['country'] == 'United States of America']
biden_df.shape

(153595, 21)

In [5]:
# biden_pd = pd.read_csv('../data/hashtag_joebiden.csv', engine='python')
biden_df = biden_pd[biden_pd['country'] == 'United States of America']
biden_df.head(5)
biden_df.shape

(153595, 21)

In [6]:
def clean_dates(df):    
    datetimes = []
    for date in df['created_at']:
        year = int(date[0:4])
        month = int(date[5:7])
        day = int(date[8:10])
        
        datetimes.append(datetime.datetime(year, month, day))
    
    df['created_datetime'] = datetimes
    return df

In [7]:
trump_df = clean_dates(trump_df)
trump_df_pre = trump_df[trump_df['created_datetime'] < datetime.datetime(2020, 11, 3)]
trump_df_pre.shape

C:\Users\CK\Anaconda3\lib\site-packages\ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # Remove the CWD from sys.path while we load stuff.


(116290, 22)

In [8]:
biden_df = clean_dates(biden_df)
biden_df_pre = biden_df[biden_df['created_datetime'] < datetime.datetime(2020, 11, 3)]
biden_df_pre.shape

C:\Users\CK\Anaconda3\lib\site-packages\ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # Remove the CWD from sys.path while we load stuff.


(89399, 22)

In [9]:
def clean_tweets(tweets):
    pattern = r'''(?x)  # set flag to allow verbose regexps
    (?:[A-Z]\.)+        # abbreviations, e.g. U.S.A.
    |\w+(?:[-']\w+)*    # words with optional internal hyphens
    |\$?\d+(?:\.\d+)?   # currency, e.g. $12.80 
    |\.\.\.             # elipses
    |[.,;"'?()-_`]      # these are separate tokens
    '''
    string_tweets = " ".join(tweets)
    tokenized_raw = " ".join(nltk.regexp_tokenize(string_tweets, pattern))
    tokenized_raw = tokenize.sent_tokenize(tokenized_raw)

    #normalizing text
    # Remove punctuations
    nopunct = []
    for sent in tokenized_raw:
        a = [w for w in sent.split() if w not in string.punctuation]
        nopunct.append(" ".join(a))
        
    # Word tokenize
    tokens = [nltk.word_tokenize(sent) for sent in nopunct]
    return(tokens)


In [19]:
trump_tokens = clean_tweets(trump_df_pre.tweet.tolist())
trump_unique_words = list(set([item for sublist in trump_tokens for item in sublist]))
trump_model = gensim.models.Word2Vec(trump_tokens, min_count=1, size=70, window=5)
trump_vector_list = trump_model[trump_unique_words]
print(np.array(trump_vector_list).shape)

C:\Users\CK\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  after removing the cwd from sys.path.


(206094, 70)


In [20]:
biden_tokens = clean_tweets(biden_df_pre.tweet.tolist())
biden_unique_words = list(set([item for sublist in biden_tokens for item in sublist]))
biden_model = gensim.models.Word2Vec(biden_tokens, min_count=1, size=70, window=5)
biden_vector_list = biden_model[biden_unique_words]
print(np.array(biden_vector_list).shape)

C:\Users\CK\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  after removing the cwd from sys.path.


(160456, 70)


In [21]:
trump_X = np.ones(70)
for word in trump_unique_words:
    trump_X = np.vstack((trump_X, trump_model[word]))
trump_X = trump_X[1:, :]

C:\Users\CK\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  This is separate from the ipykernel package so we can avoid doing imports until


In [22]:
biden_X = np.ones(70)
for word in biden_unique_words:
    biden_X = np.vstack((biden_X, biden_model[word]))
biden_X = biden_X[1:, :]

C:\Users\CK\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  This is separate from the ipykernel package so we can avoid doing imports until


In [28]:
biden_X_df = pd.DataFrame(data=biden_X)
biden_X_df['word'] = biden_unique_words
biden_X_df.to_csv('biden_X.csv')

In [29]:
trump_X_df = pd.DataFrame(data=trump_X)
trump_X_df['word'] = trump_unique_words
trump_X_df.to_csv('trump_X.csv')

In [30]:
kmeans_biden = KMeans(n_clusters = 2, random_state=0)
kmeans_biden.fit(biden_X)
biden_X_df['label'] = kmeans_biden.labels_

In [31]:
def generate_polarity_scores(df):
    tweets = df['tweet']
    punct_re = r'[^(\w)#!?@]'
    analyzer = SentimentIntensityAnalyzer()
    clean_tweets = tweets.str.replace(punct_re, ' ', regex=True).to_list()

    polarity_list = []
    for tweet in clean_tweets:
        vs = analyzer.polarity_scores(tweet)
        vs['tweet'] = tweet
        polarity_list.append(vs)
        
    new_df = pd.DataFrame(polarity_list)
    new_df['state'] = df['state']
    new_df = new_df[['tweet', 'state', 'compound', 'pos', 'neg', 'neu']].sort_values(by=['compound'], ascending=False)
    return new_df

In [ ]:
trump_polarity_df = generate_polarity_scores(trump_df_pre)
trump_polarity_df.head(10)

In [32]:
biden_polarity_df = generate_polarity_scores(biden_df_pre)
biden_polarity_df.head(10)

,tweet,state,compound,pos,neg,neu
69218,TIE SCENARIOS 2016 map but Biden wins AZ ...,NaN,0.9944,0.520,0.000,0.480
82725,#Trump says he loves #Texas But #Biden loves ...,NaN,0.9911,0.605,0.000,0.395
21681,@DrBiden And everything must go! #joebiden se...,California,0.9901,0.537,0.000,0.463
83839,@lindyli Wow!!! Lindy this is so amazing awe...,NaN,0.9876,0.588,0.000,0.412
17882,@ThatEricAlper I have a friend @JoeBiden! I t...,South Carolina,0.9873,0.487,0.000,0.513
46171,@JoeBiden HA HA HA HA HA!!! ðŸ ðŸ ðŸ ðŸ ðŸ...,NaN,0.9870,0.457,0.068,0.475
8789,@DustinGrowick My daughters!! I m a widowed mo...,District of Columbia,0.9864,0.501,0.044,0.455
58860,A Precious Human life by HH #DalaiLama hangs i...,California,0.9860,0.387,0.000,0.613
16822,It s like taking a guess when the only answer ...,District of Columbia,0.9855,0.475,0.000,0.525
6860,@PSU_Blaze Oh but it did The separate situati...,NaN,0.9847,0.477,0.060,0.463


In [33]:
def label_polarity(df):
    labels = []
    
    for index, row in df.iterrows():
        score = row['pos'] - row['neg']
        if score > 0.1:
            labels.append(1)
        elif score < -0.1:
            labels.append(0)
        else:
            labels.append(-1)
    df['label'] = labels
    df.sort_values(by = 'label')
    return df

In [34]:
biden_polarity_df = label_polarity(biden_polarity_df)
biden_polarity_df.head(10)

,tweet,state,compound,pos,neg,neu,label
69218,TIE SCENARIOS 2016 map but Biden wins AZ ...,NaN,0.9944,0.520,0.000,0.480,1
82725,#Trump says he loves #Texas But #Biden loves ...,NaN,0.9911,0.605,0.000,0.395,1
21681,@DrBiden And everything must go! #joebiden se...,California,0.9901,0.537,0.000,0.463,1
83839,@lindyli Wow!!! Lindy this is so amazing awe...,NaN,0.9876,0.588,0.000,0.412,1
17882,@ThatEricAlper I have a friend @JoeBiden! I t...,South Carolina,0.9873,0.487,0.000,0.513,1
46171,@JoeBiden HA HA HA HA HA!!! ðŸ ðŸ ðŸ ðŸ ðŸ...,NaN,0.9870,0.457,0.068,0.475,1
8789,@DustinGrowick My daughters!! I m a widowed mo...,District of Columbia,0.9864,0.501,0.044,0.455,1
58860,A Precious Human life by HH #DalaiLama hangs i...,California,0.9860,0.387,0.000,0.613,1
16822,It s like taking a guess when the only answer ...,District of Columbia,0.9855,0.475,0.000,0.525,1
6860,@PSU_Blaze Oh but it did The separate situati...,NaN,0.9847,0.477,0.060,0.463,1


In [ ]:
trump_polarity_df = label_polarity(trump_polarity_df)
trump_polarity_df.head(10)

In [ ]:
def find_most_hashtags(df, top_hashtag_num):
    pos_df = df[df['label'] == 1]
    pos_hashtag = []
    for tweet in pos_df['tweet']:
        pos_hashtag.extend(re.findall(r"(?!#trump|#biden|#Trump|#Biden|#DonaldTrump|#JoeBiden|#joebiden)#(\w+)", tweet))
        
    neg_df = df[df['label'] == 0]
    neg_hashtag = []
    for tweet in neg_df['tweet']:
        neg_hashtag.extend(re.findall(r"(?!#trump|#biden|#Trump|#Biden|#DonaldTrump|#JoeBiden|#joebiden)#(\w+)", tweet))
        
    neu_df = df[df['label'] == -1]
    neu_hashtag = []
    for tweet in neu_df['tweet']:
        neu_hashtag.extend(re.findall(r"(?!#trump|#biden|#Trump|#Biden|#DonaldTrump|#JoeBiden|#joebiden)#(\w+)", tweet))
    
    hashtags = []
    hashtags.append(Counter(pos_hashtag).most_common(top_hashtag_num))
    hashtags.append(Counter(neg_hashtag).most_common(top_hashtag_num))
    hashtags.append(Counter(neu_hashtag).most_common(top_hashtag_num))
    
    return hashtags

In [ ]:
trump_hashtags = find_most_hashtags(trump_polarity_df, 5)
biden_hashtags = find_most_hashtags(biden_polarity_df, 5)
print(trump_hashtags)
print(biden_hashtags)

In [ ]:
#what kind of groups would clustering sort them into
